In [244]:
import pandas as pd
import numpy as np
import os
import librosa
import librosa.display
import random
import shutil
import matplotlib.pyplot as plt
from scipy.io.wavfile import write

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

In [245]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [246]:
# Caminho para os dados no Google Drive
file_path_real = '/content/drive/MyDrive/amostra_FOR/real'
file_path_fake = '/content/drive/MyDrive/amostra_FOR/fake'

In [247]:
# Listar os arquivos e subpastas
print(os.listdir(file_path_real))
print(os.listdir(file_path_fake))

['file41.wav', 'file44.wav', 'file37.wav', 'file42.wav', 'file43.wav', 'file38.wav', 'file40.wav', 'file4.wav', 'file39.wav', 'file3.wav', 'file36.wav', 'file30.wav', 'file32.wav', 'file34.wav', 'file31.wav', 'file33.wav', 'file29.wav', 'file35.wav', 'file28.wav', 'file25.wav', 'file24.wav', 'file19.wav', 'file21.wav', 'file22.wav', 'file2.wav', 'file23.wav', 'file27.wav', 'file20.wav', 'file26.wav', 'file1.wav', 'file11.wav', 'file16.wav', 'file10.wav', 'file18.wav', 'file12.wav', 'file17.wav', 'file13.wav', 'file15.wav', 'file14.wav', 'file7.wav', 'file8.wav', 'file49.wav', 'file48.wav', 'file6.wav', 'file9.wav', 'file47.wav', 'file50.wav', 'file46.wav', 'file5.wav', 'file45.wav']
['file8.wav', 'file7.wav', 'file9.wav', 'file10.wav', 'file6.wav', 'file3.mp3', 'file4.wav', 'file5.mp3', 'file4.mp3', 'file3.wav', 'file1.wav', 'file5.wav', 'file2.wav']


In [248]:
# Função para processar os áudios em uma pasta
def process_audio_files(folder_path):

    for audio_file in os.listdir(folder_path):
        audio_path = os.path.join(folder_path, audio_file)

        # Carregar o áudio
        y, sr = librosa.load(audio_path, sr=None)

        # Mostrar informações do áudio
        print(f"Arquivo: {audio_file}, Duração: {len(y) / sr:.2f}s, Taxa de amostragem: {sr}Hz")

        # Plotar waveform
        plt.figure(figsize=(10, 4))
        librosa.display.waveshow(y, sr=sr)
        plt.title(f"Waveform de {audio_file}")
        plt.xlabel("Tempo (s)")
        plt.ylabel("Amplitude")
        plt.show()

In [249]:
# Processar áudios Real
#process_audio_files(file_path_real)

In [250]:
# Processar áudios Fake
#process_audio_files(file_path_fake)

In [251]:
# Função para extrair MFCCs de um arquivo
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)  # Média dos coeficientes

# Carregar dados e rótulos
data = []
labels = []

# Processar Fake
for file_name in os.listdir(file_path_fake):
    file_path = os.path.join(file_path_fake, file_name)
    features = extract_features(file_path)
    data.append(features)
    labels.append(0)  # Rótulo 0 para Fake

# Processar Real
for file_name in os.listdir(file_path_real):
    file_path = os.path.join(file_path_real, file_name)
    features = extract_features(file_path)
    data.append(features)
    labels.append(1)  # Rótulo 1 para Real

In [252]:
# Converter para DataFrame
df = pd.DataFrame(data)
df['label'] = labels

In [253]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,-282.763092,87.609459,-7.789220,29.927618,12.211132,-2.505955,-3.717391,-5.700243,-5.142696,-4.622656,-14.605258,-10.405551,-4.518505,0
1,-286.082520,96.152893,0.332997,7.762519,6.405836,1.141295,-0.186153,-2.848524,-6.321084,-9.662392,-13.558083,-7.538981,-3.196444,0
2,-301.905304,110.708092,-8.359116,0.146611,3.257944,-2.577842,0.972033,-7.275580,-3.713131,-4.957530,-11.937767,-7.162240,-3.375061,0
3,-292.760193,85.521347,-18.103905,14.233822,-1.934288,-8.300287,-15.360986,-14.333099,-0.524541,-12.453050,-2.744914,-14.212177,0.993812,0
4,-299.649109,89.557137,-4.158434,13.861190,-3.081791,2.587543,2.975845,-1.891914,-4.874742,-7.782253,-9.269706,-9.593967,-4.149051,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,-256.720123,73.813324,7.038609,53.518303,-19.797033,-0.580364,-11.983588,-15.578330,-6.098613,-0.176107,-11.477259,3.670532,-10.171000,1
59,-256.618927,84.049202,3.819647,26.094973,-11.286744,-3.361226,-16.668837,1.676700,-6.845715,2.269262,-13.792460,3.102676,-9.047886,1
60,-258.224365,83.970970,17.472389,34.816021,-12.070475,-2.597613,-17.610140,-2.063084,-12.289879,-1.229269,-16.436594,2.493302,-12.119117,1
61,-265.643036,81.611969,-0.051786,15.706508,-4.572192,6.268896,-13.187626,3.236949,-6.193376,2.356686,-19.276236,6.385146,-6.577495,1


In [254]:
# Separar características (X) e rótulos (y)
X = df.drop(columns=['label'])
y = df['label']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [255]:
# Treinando um modelo de regressão linear
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = regressor.predict(X_test)

In [256]:
# Calculando métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Erro Absoluto Médio (MAE): {mae:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")

Erro Absoluto Médio (MAE): 0.15
Coeficiente de Determinação (R²): 0.84


In [257]:
def cut_and_resize(audio, sr, target_length, num_versions=3):

    target_samples = int(target_length * sr)  # Convertendo segundos para amostras
    versions = []

    for _ in range(num_versions):
        # Se o áudio for maior, corte em uma posição aleatória
        if len(audio) > target_samples:
            start = np.random.randint(0, len(audio) - target_samples)
            resized_audio = audio[start:start + target_samples]
        else:
            # Se for menor, preencha com zeros
            resized_audio = np.pad(audio, (0, target_samples - len(audio)), 'constant')

        versions.append(resized_audio)

    return versions

def augment_and_create_combined_dir(input_dir, combined_dir, target_length=5, num_versions=3):

    # Criar diretório consolidado
    os.makedirs(combined_dir, exist_ok=True)

    # Iterar sobre os arquivos no diretório de entrada
    for file_name in os.listdir(input_dir):
        if file_name.endswith(".wav"):
            file_path = os.path.join(input_dir, file_name)

            # Copiar o arquivo original para o diretório consolidado
            shutil.copy(file_path, os.path.join(combined_dir, file_name))

            # Carrega o áudio
            audio, sr = librosa.load(file_path, sr=None)

            # Aplicar corte e redimensionamento
            augmented_versions = cut_and_resize(audio, sr, target_length, num_versions=num_versions)

            # Salvar áudios aumentados no diretório consolidado
            base_name = os.path.basename(file_name).split('.')[0]
            for i, augmented_audio in enumerate(augmented_versions):
                augmented_file_name = f"{base_name}_augmented_{i + 1}.wav"
                output_file = os.path.join(combined_dir, augmented_file_name)
                write(output_file, sr, augmented_audio)
                print(f"Áudio misturado salvo em: {output_file}")

# Caminho para os dados no Google Drive
file_path_real = '/content/drive/MyDrive/amostra_FOR/real'
file_path_fake = '/content/drive/MyDrive/amostra_FOR/fake'

# Diretórios para os dados combinados
combined_dir_real = '/content/drive/MyDrive/amostra_FOR_combined/real'
combined_dir_fake = '/content/drive/MyDrive/amostra_FOR_combined/fake'

# Processar ambas as categorias e criar diretórios consolidados com múltiplas versões
augment_and_create_combined_dir(file_path_real, combined_dir_real, target_length=5, num_versions=3)
augment_and_create_combined_dir(file_path_fake, combined_dir_fake, target_length=5, num_versions=3)

Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file41_augmented_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file41_augmented_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file41_augmented_3.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file44_augmented_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file44_augmented_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file44_augmented_3.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file37_augmented_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file37_augmented_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file37_augmented_3.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined/real/file42_augmented_1.wav
Áudio misturado salv

In [258]:
# Função para extrair MFCCs de um arquivo
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)  # Média dos coeficientes

# Diretórios de saída distintos para áudios 'real' e 'fake'
combined_dir_real = '/content/drive/MyDrive/amostra_FOR_combined/real'
combined_dir_fake = '/content/drive/MyDrive/amostra_FOR_combined/fake'

# Listas para armazenar os dados e rótulos
data = []
labels = []

# Processar Fake
for file_name in os.listdir(combined_dir_fake):
    if file_name.endswith(".wav"):  # Verificar se é um arquivo .wav
        file_path = os.path.join(combined_dir_fake, file_name)
        features = extract_features(file_path)
        data.append(features)
        labels.append(0)  # Rótulo 0 para Fake

# Processar Real
for file_name in os.listdir(combined_dir_real):
    if file_name.endswith(".wav"):  # Verificar se é um arquivo .wav
        file_path = os.path.join(combined_dir_real, file_name)
        features = extract_features(file_path)
        data.append(features)
        labels.append(1)  # Rótulo 1 para Real

In [259]:
# Converter para DataFrame
df_cut_and_resize = pd.DataFrame(data)
df_cut_and_resize['label'] = labels

In [260]:
df_cut_and_resize

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,-282.763092,87.609459,-7.789220,29.927618,12.211132,-2.505955,-3.717391,-5.700243,-5.142696,-4.622656,-14.605258,-10.405551,-4.518505,0
1,-454.433136,37.653427,-3.347708,12.862509,5.248188,-1.077028,-1.597687,-2.449892,-2.210265,-1.986759,-6.277153,-4.472173,-1.941996,0
2,-454.433136,37.653427,-3.347708,12.862509,5.248188,-1.077028,-1.597687,-2.449892,-2.210265,-1.986759,-6.277153,-4.472173,-1.941996,0
3,-454.433136,37.653427,-3.347708,12.862509,5.248188,-1.077028,-1.597687,-2.449892,-2.210265,-1.986759,-6.277153,-4.472173,-1.941996,0
4,-286.082520,96.152893,0.332997,7.762519,6.405836,1.141295,-0.186153,-2.848524,-6.321084,-9.662392,-13.558083,-7.538981,-3.196444,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,-466.106628,31.295221,0.084614,6.104418,-1.648733,2.490299,-4.949956,1.321886,-2.287387,0.974725,-7.298019,2.503658,-2.455173,1
236,-256.267517,88.543236,-10.586956,16.721491,-12.864735,7.861447,-12.899840,0.276967,-6.716938,5.733701,-19.050165,-0.664859,-10.576735,1
237,-405.679840,46.713043,-5.316601,8.820119,-6.638281,4.232179,-6.791987,0.247803,-3.577299,3.148857,-10.146777,-0.271440,-5.668361,1
238,-405.679840,46.713043,-5.316601,8.820119,-6.638281,4.232179,-6.791987,0.247803,-3.577299,3.148857,-10.146777,-0.271440,-5.668361,1


In [261]:
# Separar características (X) e rótulos (y)
X = df_cut_and_resize.drop(columns=['label'])
y = df_cut_and_resize['label']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [262]:
# Treinando um modelo de regressão linear
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = regressor.predict(X_test)

In [263]:
# Calculando métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Erro Absoluto Médio (MAE): {mae:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")

Erro Absoluto Médio (MAE): 0.15
Coeficiente de Determinação (R²): 0.76


In [264]:
def mix_with_other_audio(audio, sr, input_dir, num_mixes=2):
    """
    Mistura o áudio fornecido com outros áudios do diretório de entrada.
    """
    mixed_versions = []
    audio_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

    for _ in range(num_mixes):
        try:
            # Escolher um arquivo aleatório para mistura
            other_file = random.choice(audio_files)
            other_path = os.path.join(input_dir, other_file)

            # Carregar o áudio a ser misturado
            other_audio, _ = librosa.load(other_path, sr=sr)

            # Ajustar o tamanho do áudio misturado
            min_length = min(len(audio), len(other_audio))
            trimmed_audio = audio[:min_length]
            trimmed_other_audio = other_audio[:min_length]

            # Misturar os áudios com pesos aleatórios
            weight_original = random.uniform(0.5, 0.9)  # Peso para o áudio original
            weight_other = 1.0 - weight_original  # Peso complementar para o outro áudio
            mixed_audio = (weight_original * trimmed_audio) + (weight_other * trimmed_other_audio)

            # Normalizar e garantir que o áudio misturado esteja no intervalo [-1, 1]
            mixed_audio = mixed_audio / np.max(np.abs(mixed_audio))  # Normalizar
            mixed_audio = np.clip(mixed_audio, -1.0, 1.0)  # Garantir que os valores fiquem dentro do intervalo

            mixed_versions.append(mixed_audio)
        except Exception as e:
            print(f"Erro ao processar o arquivo {other_file}: {e}")

    return mixed_versions

def generate_mixed_data(input_dir, output_dir_fake, output_dir_real, num_mixes=2):
    """
    Gera versões misturadas de áudios e salva em subpastas 'fake' e 'real'.
    """
    # Criar as pastas 'fake' e 'real' para salvar os áudios misturados
    os.makedirs(output_dir_fake, exist_ok=True)
    os.makedirs(output_dir_real, exist_ok=True)
    print(f"Diretórios '{output_dir_fake}' e '{output_dir_real}' criados com sucesso.")

    for file_name in os.listdir(input_dir):
        if file_name.endswith(".wav"):
            try:
                file_path = os.path.join(input_dir, file_name)

                # Carregar o áudio original
                audio, sr = librosa.load(file_path, sr=None)

                # Aplicar mistura com outros áudios do mesmo diretório
                mixed_versions = mix_with_other_audio(audio, sr, input_dir, num_mixes=num_mixes)

                # Salvar versões misturadas nos diretórios 'fake' ou 'real'
                base_name = os.path.basename(file_name).split('.')[0]
                for i, mixed_audio in enumerate(mixed_versions):
                    mixed_file_name = f"{base_name}_mixed_{i + 1}.wav"
                    if 'fake' in file_name.lower():
                        output_file = os.path.join(output_dir_fake, mixed_file_name)
                    else:
                        output_file = os.path.join(output_dir_real, mixed_file_name)

                    write(output_file, sr, (mixed_audio * 32767).astype(np.int16))  # Converter para int16
                    print(f"Áudio misturado salvo em: {output_file}")
            except Exception as e:
                print(f"Erro ao processar o arquivo {file_name}: {e}")

# Exemplos de diretórios de entrada e saída
redirected_real = '/content/drive/MyDrive/amostra_FOR_combined/real'
redirected_fake = '/content/drive/MyDrive/amostra_FOR_combined/fake'

# Diretórios de saída distintos para áudios 'real' e 'fake'
output_dir_real_mixed = '/content/drive/MyDrive/amostra_FOR_combined_mixed/real'
output_dir_fake_mixed = '/content/drive/MyDrive/amostra_FOR_combined_mixed/fake'

# Gerar dados misturados a partir dos áudios redirecionados
generate_mixed_data(redirected_real, output_dir_fake_mixed, output_dir_real_mixed, num_mixes=2)
generate_mixed_data(redirected_fake, output_dir_fake_mixed, output_dir_real_mixed, num_mixes=2)

Diretórios '/content/drive/MyDrive/amostra_FOR_combined_mixed/fake' e '/content/drive/MyDrive/amostra_FOR_combined_mixed/real' criados com sucesso.
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file41_mixed_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file41_mixed_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file41_augmented_1_mixed_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file41_augmented_1_mixed_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file41_augmented_2_mixed_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file41_augmented_2_mixed_2.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file41_augmented_3_mixed_1.wav
Áudio misturado salvo em: /content/drive/MyDrive/amostra_FOR_combined_mixed/real/file41_augm

In [265]:
# Função para extrair MFCCs de um arquivo
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)  # Média dos coeficientes

# Diretórios de saída distintos para áudios 'real' e 'fake'
output_dir_real_mixed = '/content/drive/MyDrive/amostra_FOR_combined_mixed/real'
output_dir_fake_mixed = '/content/drive/MyDrive/amostra_FOR_combined_mixed/fake'

# Listas para armazenar os dados e rótulos
data = []
labels = []

# Processar Fake
for file_name in os.listdir(output_dir_fake_mixed):
    if file_name.endswith(".wav"):  # Verificar se é um arquivo .wav
        file_path = os.path.join(output_dir_fake_mixed, file_name)
        features = extract_features(file_path)
        data.append(features)
        labels.append(0)  # Rótulo 0 para Fake

# Processar Real
for file_name in os.listdir(output_dir_real_mixed):
    if file_name.endswith(".wav"):  # Verificar se é um arquivo .wav
        file_path = os.path.join(output_dir_real_mixed, file_name)
        features = extract_features(file_path)
        data.append(features)
        labels.append(1)  # Rótulo 1 para Real

In [266]:
# Converter para DataFrame
df_mix_with_other_audio = pd.DataFrame(data)
df_mix_with_other_audio['label'] = labels

In [267]:
df_mix_with_other_audio

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,-177.589066,79.727631,10.958622,27.804356,-9.359008,-10.027782,-17.362972,-3.238060,-14.149158,-1.687896,-17.452269,0.679679,-10.532807,1
1,-198.995712,73.641495,19.341797,43.502491,-11.092772,-3.114367,-9.918315,-11.143425,-11.678988,-2.881697,-15.459588,3.430488,-10.296252,1
2,-296.041046,56.980854,13.555696,24.170231,-6.890651,-6.024087,-8.290423,-2.614443,-11.023004,-0.539579,-11.084576,1.433476,-5.921502,1
3,-294.352478,57.460896,10.865855,28.539330,-8.421099,-3.949693,-6.488036,-5.243396,-7.765354,-2.221822,-9.592875,2.783510,-5.479051,1
4,-310.015717,56.168690,8.934524,17.947067,-6.516771,-6.083009,-9.652474,-1.404714,-6.914879,-0.502910,-10.065600,2.050548,-6.715950,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,-135.134415,99.219955,-5.428480,36.247055,-16.201756,7.283477,-19.553928,-11.789997,-10.538408,-0.235863,-21.941797,-0.062915,-17.795111,1
396,-337.393341,54.563576,-4.107300,15.759519,-15.770796,2.752342,-4.436332,-8.431996,-5.911708,-0.086507,-10.934151,1.113958,-7.680728,1
397,-178.028473,108.665077,-13.638209,22.304359,-19.992956,8.611423,-15.049358,-8.766487,-8.071793,4.335513,-17.026834,-2.009461,-13.629210,1
398,-297.155151,65.589600,3.692766,24.187992,-14.287136,4.793280,-7.385073,-8.519858,-4.138279,0.417238,-12.796315,2.518821,-10.974799,1


In [268]:
# Separar características (X) e rótulos (y)
X = df_mix_with_other_audio.drop(columns=['label'])
y = df_mix_with_other_audio['label']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [269]:
# Treinando um modelo de regressão linear
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = regressor.predict(X_test)

In [270]:
# Calculando métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Erro Absoluto Médio (MAE): {mae:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")

Erro Absoluto Médio (MAE): 0.00
Coeficiente de Determinação (R²): 1.00
